# Analyze dDEMs as a function of land cover type, canopy height, and terrain parameters

In [ ]:
import os
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr
import numpy as np
import xdem
import seaborn as sns
import pandas as pd
import geopandas as gpd
import xrspatial

In [ ]:
# pipeline inputs
data_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/'
refdem_fn = os.path.join(data_dir, 'study-sites', 'MCS', 'refdem', 'MCS_REFDEM_WGS84.tif')
chm_fn = os.path.join(data_dir, 'study-sites', 'MCS', 'refdem', 'chm_mcs_1m.tif')

# pipeline outputs
job_dir = os.path.join(data_dir, 'snow_depth_maps', 'MCS_20240420-1_refbareearth_SMRFfiltered')
ddem_fn = os.path.join(job_dir, 'coreg_final_diff', 'ddem.tif')
roads_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'roads_mask.tif')
snow_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'snow_mask.tif')
ss_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'stable_surfaces_mask.tif')
trees_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'trees_mask.tif')

# output folder
out_dir = os.path.join(job_dir, 'ddem_analysis')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

## Load reference DEM

In [ ]:
# Reference DEM
refdem = xr.open_dataset(refdem_fn)
refdem = refdem.rename_vars({'band_data': 'elevation'}).squeeze()

# Function to load and adjust other files
def load_adjust_file(fn, refdem):
    file = rxr.open_rasterio(fn).squeeze()
    crs = file.rio.crs
    file = xr.where(file==file.attrs['_FillValue'], np.nan, file)
    file = file.sel(x=refdem.x.data, y=refdem.y.data, method='nearest')
    file = file.rio.write_crs(crs)
    return file


## dDEM vs. canopy height

In [ ]:
# Load input files
ddem = load_adjust_file(ddem_fn, refdem)
chm = load_adjust_file(chm_fn, refdem)

fig_fn = os.path.join(out_dir, 'ddem_vs_chm.png')
if not os.path.exists(fig_fn):

    # Create pandas.DataFrame
    df = pd.DataFrame({'dDEM': np.ravel(ddem.data),
                       'CHM': np.ravel(chm.data)})
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.loc[df['CHM'] < 0, 'CHM'] = 0

    # Create bins for CHM
    df['CHM_bin'] = pd.cut(df['CHM'], bins=np.linspace(-1,50,52))

    # Plot
    fig, ax = plt.subplots(figsize=(14,8))
    sns.boxplot(data=df, x='CHM_bin', y='dDEM', showfliers=False, ax=ax)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_ylabel('dDEM [m]')
    ax.set_xlabel('Canopy height [m]')
    plt.show()

    # Save figure
    fig_fn = os.path.join(out_dir, 'ddem_vs_chm.png')
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)
    
else:
    print('Figure already exists, skipping.')

## dDEM vs. land cover type

In [ ]:
fig_fn = os.path.join(out_dir, 'ddem_vs_land_cover_types.png')

if not os.path.exists(fig_fn):
    # Load land cover masks
    roads_mask = load_adjust_file(roads_mask_fn, refdem)
    snow_mask = load_adjust_file(snow_mask_fn, refdem)
    ss_mask = load_adjust_file(ss_mask_fn, refdem)
    trees_mask = load_adjust_file(trees_mask_fn, refdem)

    # Create dataframe
    df = pd.DataFrame({'dDEM': np.ravel(ddem.data),
                       'roads_mask': np.ravel(roads_mask.data),
                       'snow_mask': np.ravel(snow_mask.data),
                       'ss_mask': np.ravel(ss_mask.data),
                       'trees_mask': np.ravel(trees_mask.data)})
    df.dropna(inplace=True)

    # Plot histogram
    bins = np.linspace(-10, 10, 200)
    cols = ['snow_mask', 'ss_mask', 'trees_mask', 'roads_mask'] 
    colors = [(55/255, 126/255, 184/255, 1), # snow
              (153/255, 153/255, 153/255, 1), # stable surfaces
              (77/255, 175/255, 74/255, 1), # trees
              (166/255, 86/255, 40/255, 1)] # roads
    fig, ax = plt.subplots(2, 2, figsize=(10,5))
    ax = ax.flatten()
    i=0
    for col, color in zip(cols, colors):
        df_sub = df.loc[df[col]==1, 'dDEM']
        ax[i].hist(df_sub.values, bins=bins, color=color, alpha=0.8, label=col)
        ax[i].set_xlim(-5, 5)
        ax[i].set_title(col.replace('ss_mask', 'stable surfaces').replace('_mask', ''))
        i+=1

    fig.tight_layout()
    plt.show()

    # Save figure
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)
    
else:
    print('Figure already exists, skipping.')

## dDEM vs. terrain parameters

In [ ]:
fig_fn = os.path.join(out_dir, 'ddem_vs_terrain_parameters.png')

if not os.path.exists(fig_fn):
    plt.rcParams.update({'font.sans-serif': 'Arial', 'font.size': 12})
    
    # Load refdem
    refdem = xr.open_dataset(refdem_fn)
    refdem = refdem.rename_vars({'band_data': 'elevation'}).squeeze()
    # Coarsen reference DEM to 5 m to speed things up
    dx = refdem.x.data[1] - refdem.x.data[0]
    dx_new = 5
    refdem = (refdem.coarsen(x=int(dx_new/dx), boundary='trim').mean()
                    .coarsen(y=int(dx_new/dx), boundary='trim').mean())
    # Load dDEM
    ddem = xr.open_dataset(ddem_fn).squeeze() 
    # Coarsen
    ddem = (ddem.coarsen(x=int(dx_new/dx), boundary='trim').mean()
                .coarsen(y=int(dx_new/dx), boundary='trim').mean())
    # Sample dDEM at new coords
    # ddem = ddem.sel(x=refdem.x, y=refdem.x, method='nearest')
    # Calculate terrain parameters from reference DEM
    refdem['slope'] = xrspatial.slope(refdem.elevation)
    refdem['aspect'] = xrspatial.aspect(refdem.elevation)
    # Compile all stats into dataframe
    terrain_cols = ['elevation', 'slope', 'aspect']
    stats_df = pd.DataFrame(columns=terrain_cols + ['dDEM'])
    for col in terrain_cols:
        stats_df[col] = np.ravel(refdem[col].data)
        # Create bins for column
        stats_df[col + '_bin'] = pd.cut(stats_df[col], bins=25, precision=0)
    stats_df['dDEM'] = np.ravel(ddem.band_data.data)   
    stats_df.dropna(inplace=True)
    stats_df.reset_index(drop=True, inplace=True)
    
    # Plot
    fig, ax = plt.subplots(1,3, figsize=(16,6))
    for i, col in enumerate(terrain_cols):
        sns.boxplot(stats_df, x=col + '_bin', y='dDEM', showfliers=False, ax=ax[i])
        ax[i].set_title(col)
        if i==0:
            ax[i].set_ylabel('dDEM [m]')
        ax[i].set_xlabel(col)
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=90)
        ax[i].axhline(0, color='k')
    fig.tight_layout()
    plt.show()
    # Save figure
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)
    
else:
    print('Figure already exists, skipping.')

In [ ]:
# Plot SNOTEL
snotel_fn = os.path.join(data_dir, 'study-sites', 'MCS', 'snotel', 'MCS_2020-01-01_2024-06-07_adj.csv')
snotel = pd.read_csv(snotel_fn)
snotel['datetime'] = pd.to_datetime(snotel['datetime'])

fig = plt.figure(figsize=(8,6))
plt.plot(snotel['datetime'], snotel['SWE_m'], '-', color='purple', label='SWE')
plt.plot(snotel['datetime'], snotel['SNWD_m'], '-', color='blue', label='Snow depth')
plt.axvline(np.datetime64('2024-04-20'), color='k', linestyle='--', markersize=15, label='SkySat date')
skysat_sd = snotel.loc[snotel['datetime'].dt.date == np.datetime64('2024-04-20'), 'SNWD_m'].values[0]
plt.text(np.datetime64('2024-04-15'), 2.6, f'Snow depth = {np.round(skysat_sd, 3)} m', ha='right')
plt.xlim(np.datetime64('2023-11-01'), np.datetime64('2024-07-01'))
plt.ylabel('meters')
plt.grid()
plt.legend(loc='best')
plt.show()

fig_fn = os.path.join(data_dir, 'study-sites', 'MCS', 'snotel', 'SNWD_SWE_timeseries.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
# Plot SNOTEL location on orthoimage
from shapely.wkt import loads

ortho_fn = os.path.join(data_dir, 'study-sites', 'MCS', '20240420', 'MCS_20240420_4band_orthomosaic.tif')
ortho = xr.open_dataset(ortho_fn)
crs = ortho.rio.crs
ortho = xr.where(ortho==0, np.nan, ortho / 1e4)
ortho = ortho.rio.write_crs(crs)
snotel_info_fn = os.path.join(data_dir, 'study-sites', 'MCS', 'snotel', 'MCS_SNOTEL_site_info.csv')
snotel_info = pd.read_csv(snotel_info_fn)
snotel_info['geometry'] = snotel_info['geometry'].apply(loads)
snotel_info = gpd.GeoDataFrame(snotel_info, geometry='geometry', crs='EPSG:4326')
snotel_info = snotel_info.to_crs(ortho.rio.crs)

fig = plt.figure(figsize=(6,6))
plt.imshow(np.dstack([ortho.isel(band=2).band_data.data, 
                      ortho.isel(band=1).band_data.data, 
                      ortho.isel(band=0).band_data.data]) * 0.5,
           extent=(np.min(ortho.x)/1e3, np.max(ortho.x)/1e3, 
                   np.min(ortho.y)/1e3, np.max(ortho.y)/1e3))
plt.plot(snotel_info.geometry[0].coords.xy[0][0]/1e3, 
         snotel_info.geometry[0].coords.xy[1][0]/1e3, '*', 
         markerfacecolor='yellow', markeredgecolor='k', markersize=30)
plt.xlabel('Easting [km]')
plt.ylabel('Northing [km]')
plt.show()

fig_fn = snotel_info_fn.replace('.csv', '.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)